# Player Evaluation

Player evaluation will be conducted prior to roster design, as it weighs the overall performance of each player from each game throughout the course of a season. Total Hockey Rating (THoR) is an all-inclusive statistic rating of all NHL defensemen and forwards incidental to all on-ice events. All events of a game are documented and appointed a value determined by the probability that event generated a goal. 

### purpose of notebook:

a) generate a variable that will show the time difference between a goal and all events that happened prior.


b) keep only events that happened 20 seconds prior to a goal.

c) group events by goal number to count the occurance of each event prior to a goal.

d) sum by event type to display the incidence of each event in two games.

e) establish the impact of each event on a goal.

f) deterime if events have a positive or negative impact on each team.

g) assign values to players based on their participation in events that led to a goal.

##  import modules

In [106]:
import sys
import os
import pandas as pd
import numpy as np
import datetime, time
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from pylab import hist, show
import scipy

## import data frame

The merged data frame created in the roster_design_stephanos notebook is imported and used for player evaluation.

In [107]:
dm = pd.read_csv('pbpmerge.csv')

## drop unnamed column (irrelevant)

In [108]:
dm = dm.drop('Unnamed: 0', axis=1)

## a list of the columns in pbpmerge data frame

In [109]:
dm.columns

Index(['Season', 'GameNumber', 'EventNumber', 'Period', 'AdvantageType',
       'EventTimeFromZero', 'EventTimeFromTwenty', 'EventType', 'EventDetail',
       'VPlayer1', 'VPlayer1Position', 'VPlayer2', 'VPlayer2Position',
       'VPlayer3', 'VPlayer3Position', 'VPlayer4', 'VPlayer4Position',
       'VPlayer5', 'VPlayer5Position', 'VPlayer6', 'VPlayer6Position',
       'HPlayer1', 'HPlayer1Position', 'HPlayer2', 'HPlayer2Position',
       'HPlayer3', 'HPlayer3Position', 'HPlayer4', 'HPlayer4Position',
       'HPlayer5', 'HPlayer5Position', 'HPlayer6', 'HPlayer6Position',
       'TeamCode', 'PlayerNumber', 'PlayerName', 'ShotType', 'Zone', 'Length',
       'ShotResult', 'ShotTeamCode', 'ShotPlayerNumber', 'ShotPlayerName',
       'WinTeamCode', 'VTeamCode', 'VPlayerNumber', 'VPlayerName', 'HTeamCode',
       'HPlayerNumber', 'HPlayerName', 'HitterTeamCode', 'HitterPlayerNumber',
       'HitterPlayerName', 'HitteeTeamCode', 'HitteePlayerNumber',
       'HitteePlayerName', 'PenaltyTeamCod

## fill in team code for all type of events

For team code column not to be missing any data, **numpy where** is used. It is a command that assigns values to team code based on the event type and the outcome of that event. 

- if an event is a faceoff, the team that won the faceoff will be assigned to 'TeamCode'. 

- if an event is a hit, the team that registered a hit will be assigned to 'TeamCode'. 

- if an event is a penalty, the team that committed the penalty will be assigned to 'TeamCode'.

In [110]:
dm['TeamCode'] = np.where(dm['EventType'] == 'FAC', dm['WinTeamCode'],
                             (np.where(dm['EventType'] == 'HIT', dm['HitterTeamCode'],
                                       (np.where(dm['EventType'] == 'PENL', dm['PenaltyTeamCode'], dm['TeamCode'])))))

## fill in home and visitor team code 

To confirm there are no missing data, home and visitor team code are filled in *backwards*. 

 - visitor team code for all events prior to a goal filled in backwards

In [111]:
dm['VTeamCode'] = dm['VTeamCode'].fillna(method='bfill')

 - home team code for all events prior to a goal filled in backwards

In [112]:
dm['HTeamCode'] = dm['HTeamCode'].fillna(method='bfill')

##  fill in variables goal number and goal time with values

Goal number and goal time values will be assigned to every event, dependent on the number of goals scored in a game and the time (from zero) they happened. Since events that occured **prior to a goal** are being examined, *fill in backwards method* is used. This will assist with the calculation of time difference between a goal and a given event.

 - goal number for all events prior to a goal filled in backwards

In [113]:
dm['GoalNumber']= dm['GoalNumber'].fillna(method='bfill')

- goal time for all events prior to a goal filled in backwards

In [114]:
dm['GoalTime'] = dm['GoalTime'].fillna(method='bfill')

## generate a variable that will calculate the time difference between goal and events

The time difference between a goal and an event is calculated as followed: 

In [115]:
dm['TBGoalandEvent'] = dm['GoalTime'] - dm['EventTimeFromZero']

## keep only events that happened 20 seconds prior to goal

The playler evaluation model uses only events that happened 20 seconds prior to a goal. If the time between a goal and an event exceeds 20 seconds, the event will not be included in the dataframe. Thus:

In [116]:
dm = dm[dm['TBGoalandEvent'] <= 20]

In [117]:
dm = dm[dm['TBGoalandEvent'] >= 0]

## create a column that will show the total observations for two games

the data is grouped by season to count the total occurance of events that happened 20 seconds prior to a goal, in the first two games of the season

In [118]:
dm['Counts'] = dm.groupby('Season')['EventType'].transform('count')

## create columns for each type of event and assign values to determine the impact they have on a goal

Values are appointed to eight even strengths events: block shot, face-off, shot on goal, missed shot, penalty, hit, takeaway, giveaway and goal.

 - create a column that assigns a value of 1 to block events and a value of 0 to every non block event

In [119]:
dm['block'] = np.where(dm['EventType'] == 'BLOCK', 1, 0)

 - create a column that assigns a value of 1 to faceoff events and a value of 0 to every non faceoff event

In [120]:
dm['faceoff'] = np.where(dm['EventType'] == 'FAC', 1, 0)

 - create a column that assigns a value of 1 to giveaway events and a value of 0 to every non giveaway event

In [121]:
dm['giveaway'] = np.where(dm['EventType'] == 'GIVE', 1, 0)

- create a column that assigns a value of 1 to goal events and a value of 0 to every non goal event

In [122]:
dm['goal'] = np.where(dm['EventType'] == 'GOAL', 1, 0)

- create a column that assigns a value of 1 to hit events and a value of 0 to every non hit event

In [123]:
dm['hit'] = np.where(dm['EventType'] == 'HIT', 1, 0)

- create a column that assigns a value of 1 to miss events and a value of 0 to every non miss shot event

In [124]:
dm['miss'] = np.where(dm['EventType'] == 'MISS', 1, 0)

 - create a column that assigns a value of 1 to penalty events and a value of 0 to every non penalty events

In [125]:
dm['penalty'] = np.where(dm['EventType'] == 'PENL', 1, 0)

- create a column that assigns a value of 1 to shot events and a value of 0 to every non shot events

In [126]:
dm['shot'] = np.where(dm['EventType'] == 'SHOT', 1, 0)

 - create a column that assigns a value of 1 to takeaway events and a value of 0 to non takeaway events

In [127]:
dm['takeaway'] = np.where(dm['EventType'] == 'TAKE', 1, 0)

## display of each event leading to a goal for two games

The below table lists the occurance of each event type prior to each goal.

In [128]:
dy = dm.groupby(['Season','GameNumber', 'GoalNumber', 'EventType']).size()

## create a variable that will display the value of each event 

All events that happened 20 seconds prior to a goal are counted. The **mean** is used to establish the impact each event has on a goal.

Fist step is to determine if an event has a positive or negative impact on a goal:

 - giveaway has a negative impact on the team that lost possession.

 - faceoff has a positive impact on the team that won the faceoff and a negative impact on the team that lost. 

- hit has a positive impact for the team that delivered the hit and a negative impact on the team that received the hit.

- penalty has a positive impact on the team that drew the penalty and a negative impact on the team serving. 

 - takeaway has a positive impact on the team that stole the puck and gained possession.

In [129]:
dm['eventvalue'] = np.where(dm['EventType'] == 'BLOCK', dm['block'].mean(),
                             (np.where(dm['EventType'] == 'FAC', dm['faceoff'].mean(),
                                       (np.where(dm['EventType'] == 'GIVE', -(dm['giveaway'].mean()),
                                                 (np.where(dm['EventType'] == 'GOAL', dm['goal'].mean(),
                                                          (np.where(dm['EventType'] == 'HIT', dm['hit'].mean(),
                                                                   (np.where(dm['EventType'] == 'MISS', dm['miss'].mean(),
                                                                            (np.where(dm['EventType'] == 'PENL', -(dm['penalty'].mean()),
                                                                                     (np.where(dm['EventType'] == 'SHOT', dm['shot'].mean(),
                                                                                              (np.where(dm['EventType'] == 'TAKE', dm['takeaway'].mean(), 0)))))))))))))))))

All event types have been assigned a value based on the impact they had on a goal. 

##  create event value for home and visitor teams

Each event has an effect on both home and visitor team. An event that has a positive impact on the home team will have a negative impact on the visitor team. Similarly, an event that has a negative effect on the home team, will have a positive effect on the visitor team.

- If an event has a positive impact on the **home team**, the mean will be positive. If an event has a negative impact on the home team, the mean will be negative.

In [130]:
dm['heventvalue'] = np.where(dm['TeamCode'] == dm['HTeamCode'], dm['eventvalue'], -(dm['eventvalue']))

- If an event has a positive impact on the **visitor team**, the mean will be positive. If an event has a negative impact on the home team, the mean will be negative.

In [131]:
dm['veventvalue'] = np.where(dm['TeamCode'] == dm['VTeamCode'], dm['eventvalue'], -(dm['eventvalue']))

## assign values to each player 

The value of an event is assigned to all players that were on ice, a total of 12 players (6 per team). The overall contribution of each player is the total (sum) of events they participated in. 

### a) overall contribution of each player from the visitor team in all (6) positions 

Group data frame by season, visitor team code and visitor player position to seperate players that play in the same position. 

- create variable that evaluates the overall contribution of each player from the visitor team that is listed as "VPlayer1"

In [132]:
dm['vp1'] = dm.groupby(['Season', 'VTeamCode', 'VPlayer1'])['veventvalue'].transform('sum')

- create variable that evaluates each player from the visitor team that is listed as "VPlayer2"

In [133]:
dm['vp2'] = dm.groupby(['Season', 'VTeamCode', 'VPlayer2'])['veventvalue'].transform('sum')

- create variable that evaluates each player from the visitor team that is listed as "VPlayer3"

In [134]:
dm['vp3'] = dm.groupby(['Season', 'VTeamCode', 'VPlayer3'])['veventvalue'].transform('sum')

- create variable that evaluates each player from the visitor team that is listed as "VPlayer4"

In [135]:
dm['vp4'] = dm.groupby(['Season', 'VTeamCode', 'VPlayer4'])['veventvalue'].transform('sum')

- create variable that evaluates each player from the visitor team that is listed as "VPlayer5"

In [136]:
dm['vp5'] = dm.groupby(['Season', 'VTeamCode', 'VPlayer5'])['veventvalue'].transform('sum')

- create variable that evaluates each player from the visitor team that is listed as "VPlayer6"

In [137]:
dm['vp6'] = dm.groupby(['Season', 'VTeamCode', 'VPlayer6'])['veventvalue'].transform('sum')

### b) overall contribution of each player from the home team in all (6) positions

Group data frame by season, home team code and home player position to seperate players that play in the same position. 

- create variable that evaluates each player from the home team that is listed as "HPlayer1"

In [138]:
dm['hp1'] = dm.groupby(['Season', 'HTeamCode', 'HPlayer1'])['heventvalue'].transform('sum')

- create variable that evaluates each player from the home team that is listed as "HPlayer2"

In [139]:
dm['hp2'] = dm.groupby(['Season', 'HTeamCode', 'HPlayer2'])['heventvalue'].transform('sum')

- create variable that evaluates each player from the home team that is listed as "HPlayer3"

In [140]:
dm['hp3'] = dm.groupby(['Season', 'HTeamCode', 'HPlayer3'])['heventvalue'].transform('mean')

- create variable that evaluates each player from the home team that is listed as "HPlayer4"

In [141]:
dm['hp4'] = dm.groupby(['Season', 'HTeamCode', 'HPlayer4'])['heventvalue'].transform('sum')

- create variable that evaluates each player from the home team that is listed as "HPlayer5"

In [142]:
dm['hp5'] = dm.groupby(['Season', 'HTeamCode', 'HPlayer5'])['heventvalue'].transform('sum')

- create variable that evaluates each player from the home team that is listed as "HPlayer6"

In [143]:
dm['hp6'] = dm.groupby(['Season', 'HTeamCode', 'HPlayer6'])['heventvalue'].transform('sum')

## players that play in multiple positions

Throughout the duration of a game, a player may change position. As mentioned above, the overall impact of a given player is the total (sum) of events he participated in. To properly measure each player's contribution, a cross examination per position must be applied. 

- **a) cross examine each position for visitor team:**

**visitor player 1**

- if visitor player in position 1 played in any games at position 2, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 1 and 2. If visitor player in position 1, didn't play in any other position, the total value of events in position 1 he participated in, are listed.

In [144]:
dm['tvp1'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer1'] == dm['VPlayer2']), dm['vp1'] + dm['vp2'], dm['vp1'])
   

- if visitor player in position 1 played in any games at position 3, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 1 and 3. If visitor player in position 1, didn't play in any other position, the total value of events in position 1 he participated in, are listed.

In [145]:
dm['tvp1'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer1'] == dm['VPlayer3']), dm['vp1'] + dm['vp3'], dm['vp1'])
   

- if visitor player in position 1 played in any games at position 4, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 1 and 4. If visitor player in position 1, didn't play in any other position, the total value of events in position 1 he participated in, are listed.

In [146]:
dm['tvp1'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer1'] == dm['VPlayer4']), dm['vp1'] + dm['vp4'], dm['vp1'])
   

- if visitor player in position 1 played in any games at position 5, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 1 and 5. If visitor player in position 1, didn't play in any other position, the total value of events in position 1 he participated in, are listed.

In [147]:
dm['tvp1'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer1'] == dm['VPlayer5']), dm['vp1'] + dm['vp5'], dm['vp1'])
   

- if visitor player in position 1 played any games in position 6 throughtout the season, the total of events he was on the ice for are calculated with the use of **np.where**. A player might play in position 6 (goaltender position) when his team is trailing in the final minutes of a game and they decide to pull the goaltender to add an additonal skater. It is the sum of events for position 1 and 6.

In [148]:
dm['tvp1'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer1'] == dm['VPlayer6']), dm['vp1'] + dm['vp6'], dm['vp1'])

**visitor player 2**

- if visitor player in position 2 played in any games at position 1, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 2 and 1. If visitor player in position 2, didn't play in any other position, the total value of events in position 2 he participated in, are listed.

In [149]:
dm['tvp2'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer2'] == dm['VPlayer1']), dm['vp2'] + dm['vp1'], dm['vp2'])

- if visitor player in position 2 played in any games at position 3, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 2 and 3. If visitor player in position 2, didn't play in any other position, the total value of events in position 2 he participated in, are listed.

In [150]:
dm['tvp2'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer2'] == dm['VPlayer3']), dm['vp2'] + dm['vp3'], dm['vp2'])

- if visitor player in position 2 played in any games at position 4, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 2 and 4. If visitor player in position 2, didn't play in any other position, the total value of events in position 2 he participated in, are listed.

In [151]:
dm['tvp2'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer2'] == dm['VPlayer4']), dm['vp2'] + dm['vp4'], dm['vp2'])

- if visitor player in position 2 played in any games at position 5, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 2 and 5. If visitor player in position 2, didn't play in any other position, the total value of events in position 2 he participated in, are listed.

In [152]:
dm['tvp2'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer2'] == dm['VPlayer1']), dm['vp2'] + dm['vp5'], dm['vp2'])

- if visitor player in position 2 played any games in position 6 throughtout the season, the total of events he was on the ice for are calculated with the use of **np.where**. A player might play in position 6 (goaltender position) when his team is trailing in the final minutes of a game and they decide to pull the goaltender to add an additonal skater. It is the sum of  events for position 2 and 6.

In [153]:
dm['tvp2'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer2'] == dm['VPlayer1']), dm['vp2'] + dm['vp6'], dm['vp2'])

**visitor player 3**

- if visitor player in position 3 played in any games at position 1, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 3 and 1. If visitor player in position 3, didn't play in any other position, the total value of events in position 3 he participated in, are listed.

In [154]:
dm['tvp3'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer3'] == dm['VPlayer1']), dm['vp3'] + dm['vp1'], dm['vp3'])

- if visitor player in position 3 played in any games at position 2, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 3 and 2. If visitor player in position 3, didn't play in any other position, the total value of events in position 3 he participated in, are listed.

In [155]:
dm['tvp3'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer3'] == dm['VPlayer2']), dm['vp3'] + dm['vp2'], dm['vp3'])

- if visitor player in position 3 played in any games at position 4, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 3 and 4. If visitor player in position 3, didn't play in any other position, the total value of events in position 3 he participated in, are listed.

In [156]:
dm['tvp3'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer3'] == dm['VPlayer4']), dm['vp3'] + dm['vp4'], dm['vp3'])

- if visitor player in position 3 played in any games at position 5, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 3 and 5. If visitor player in position 3, didn't play in any other position, the total value of events in position 3 he participated in, are listed.

In [157]:
dm['tvp3'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer3'] == dm['VPlayer5']), dm['vp3'] + dm['vp5'], dm['vp3'])

- if visitor player in position 3 played any games in position 6 throughtout the season, the total of events he was on the ice for are calculated with the use of **np.where**. A player might play in position 6 (goaltender position) when his team is trailing in the final minutes of a game and they decide to pull the goaltender to add an additonal skater. It is the sum of  events for position 3 and 6.

In [158]:
dm['tvp3'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer3'] == dm['VPlayer6']), dm['vp3'] + dm['vp6'], dm['vp3'])

**visitor player 4**

- if visitor player in position 4 played in any games at position 1, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 4 and 1. If visitor player in position 4, didn't play in any other position, the total value of events in position 4 he participated in, are listed.

In [159]:
dm['tvp4'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer4'] == dm['VPlayer1']), dm['vp4'] + dm['vp1'], dm['vp4'])

- if visitor player in position 4 played in any games at position 2, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 4 and 2. If visitor player in position 4, didn't play in any other position, the total value of events in position 4 he participated in, are listed.

In [160]:
dm['tvp4'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer4'] == dm['VPlayer2']), dm['vp4'] + dm['vp2'], dm['vp4'])

- if visitor player in position 4 played in any games at position 3, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 4 and 3. If visitor player in position 4, didn't play in any other position, the total value of events in position 4 he participated in, are listed.

In [161]:
dm['tvp4'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer4'] == dm['VPlayer3']), dm['vp4'] + dm['vp3'], dm['vp4'])

- if visitor player in position 4 played in any games at position 5, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 4 and 5. If visitor player in position 4, didn't play in any other position, the total value of events in position 4 he participated in, are listed.

In [162]:
dm['tvp4'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer4'] == dm['VPlayer5']), dm['vp4'] + dm['vp5'], dm['vp4'])

- if visitor player in position 4 played any games in position 6 throughtout the season, the total of events he was on the ice for are calculated with the use of **np.where**. A player might play in position 6 (goaltender position) when his team is trailing in the final minutes of a game and they decide to pull the goaltender to add an additonal skater. It is the sum of  events for position 4 and 6.

In [163]:
dm['tvp4'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer4'] == dm['VPlayer6']), dm['vp4'] + dm['vp6'], dm['vp4'])

**visitor position 5**

- if visitor player in position 5 played in any games at position 1, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 5 and 1. If visitor player in position 5, didn't play in any other position, the total value of events in position 5 he participated in, are listed.

In [164]:
dm['tvp5'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer5'] == dm['VPlayer1']), dm['vp5'] + dm['vp1'], dm['vp5'])


- if visitor player in position 5 played in any games at position 2, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 5 and 2. If visitor player in position 5, didn't play in any other position, the total value of events in position 5 he participated in, are listed.

In [165]:
dm['tvp5'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer5'] == dm['VPlayer2']), dm['vp5'] + dm['vp2'], dm['vp5'])


- if visitor player in position 5 played in any games at position 3, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 5 and 3. If visitor player in position 5, didn't play in any other position, the total value of events in position 5 he participated in, are listed.

In [166]:
dm['tvp5'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer5'] == dm['VPlayer3']), dm['vp5'] + dm['vp3'], dm['vp5'])


- if visitor player in position 5 played in any games at position 4, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 5 and 4. If visitor player in position 5, didn't play in any other position, the total value of events in position 5 he participated in, are listed.

In [167]:
dm['tvp5'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer5'] == dm['VPlayer4']), dm['vp5'] + dm['vp4'], dm['vp5'])


- if visitor player in position 5 played any games in position 6 throughtout the season, the total of events he was on the ice for are calculated with the use of **np.where**. A player might play in position 6 (goaltender position) when his team is trailing in the final minutes of a game and they decide to pull the goaltender to add an additonal skater. It is the sum of  events for position 5 and 6.

In [168]:
dm['tvp5'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer5'] == dm['VPlayer6']), dm['vp5'] + dm['vp6'], dm['vp5'])


**visitor player 6**

Position 6 is the goaltender position. A goalie doesn't play in any other position, therefore :

In [169]:
dm['tvp6'] = dm['vp6']

- **b) cross examine each position for home team:**

**home position 1**

- if home player in position 1 played in any games at position 2, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 1 and 2. If home player in position 1, didn't play in any other position, the total value of events in position 1 he participated in, are listed.

In [170]:
dm['thp1'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer1'] == dm['HPlayer2']), dm['hp1'] + dm['hp2'], dm['hp1'])
   

- if home player in position 1 played in any games at position 3, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 1 and 3. If home player in position 1, didn't play in any other position, the total value of events in position 1 he participated in, are listed.

In [171]:
dm['thp1'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer1'] == dm['HPlayer3']), dm['hp1'] + dm['hp3'], dm['hp1'])
   

- if home player in position 1 played in any games at position 4, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 1 and 4. If home player in position 1, didn't play in any other position, the total value of events in position 1 he participated in, are listed.

In [172]:
dm['thp1'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer1'] == dm['HPlayer4']), dm['hp1'] + dm['hp4'], dm['hp1'])
   

- if home player in position 1 played in any games at position 5, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 1 and 5. If home player in position 1, didn't play in any other position, the total value of events in position 1 he participated in, are listed.

In [173]:
dm['thp1'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer1'] == dm['HPlayer5']), dm['hp1'] + dm['hp5'], dm['hp1'])
   

- if home player in position 1 played any games in position 6 throughtout the season, the total of events he was on the ice for are calculated with the use of **np.where**. A player might play in position 6 (goaltender position) when his team is trailing in the final minutes of a game and they decide to pull the goaltender to add an additonal skater. It is the sum of  events for position 1 and 6.

In [174]:
dm['thp1'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer1'] == dm['HPlayer6']), dm['hp1'] + dm['hp6'], dm['hp1'])
   

**home player 2**

- if home player in position 2 played in any games at position 1, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 2 and 1. If home player in position 2, didn't play in any other position, the total value of events in position 2 he participated in, are listed.

In [175]:
dm['thp2'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer2'] == dm['HPlayer1']), dm['hp2'] + dm['hp1'], dm['hp2'])
   

- if home player in position 2 played in any games at position 3, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 2 and 3. If home player in position 2, didn't play in any other position, the total value of events in position 2 he participated in, are listed.

In [176]:
dm['thp2'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer2'] == dm['HPlayer3']), dm['hp2'] + dm['hp3'], dm['hp2'])
   

- if home player in position 2 played in any games at position 4, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 2 and 4. If home player in position 2, didn't play in any other position, the total value of events in position 2 he participated in, are listed.

In [177]:
dm['thp2'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer2'] == dm['HPlayer4']), dm['hp2'] + dm['hp4'], dm['hp2'])
   

- if home player in position 2 played in any games at position 5, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 2 and 5. If home player in position 2, didn't play in any other position, the total value of events in position 2 he participated in, are listed.

In [178]:
dm['thp2'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer2'] == dm['HPlayer5']), dm['hp2'] + dm['hp5'], dm['hp2'])
   

- if home player in position 2 played any games in position 6 throughtout the season, the total of events he was on the ice for are calculated with the use of **np.where**. A player might play in position 6 (goaltender position) when his team is trailing in the final minutes of a game and they decide to pull the goaltender to add an additonal skater. It is the sum of  events for position 2 and 6.

In [179]:
dm['thp2'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer2'] == dm['HPlayer6']), dm['hp2'] + dm['hp6'], dm['hp2'])
   

**home player 3**

- if home player in position 3 played in any games at position 1, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 3 and 1. If home player in position 3, didn't play in any other position, the total value of events in position 1 he participated in, are listed.

In [180]:
dm['thp3'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer3'] == dm['HPlayer1']), dm['hp3'] + dm['hp1'], dm['hp3'])
   

- if home player in position 3 played in any games at position 3, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 3 and 2. If home player in position 3, didn't play in any other position, the total value of events in position 3 he participated in, are listed.

In [181]:
dm['thp3'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer3'] == dm['HPlayer2']), dm['hp3'] + dm['hp2'], dm['hp3'])
   

- if home player in position 3 played in any games at position 5, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 3 and 4. If home player in position 3, didn't play in any other position, the total value of events in position 3 he participated in, are listed.

In [182]:
dm['thp3'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer3'] == dm['HPlayer4']), dm['hp3'] + dm['hp4'], dm['hp3'])
   

- if home player in position 3 played in any games at position 5, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 3 and 5. If home player in position 3, didn't play in any other position, the total value of events in position 3 he participated in, are listed.

In [183]:
dm['thp3'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer3'] == dm['HPlayer5']), dm['hp3'] + dm['hp5'], dm['hp3'])
   

- if home player in position 3 played any games in position 6 throughtout the season, the total of events he was on the ice for are calculated with the use of **np.where**. A player might play in position 6 (goaltender position) when his team is trailing in the final minutes of a game and they decide to pull the goaltender to add an additonal skater. It is the sum of  events for position 3 and 6.

In [184]:
dm['thp3'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer3'] == dm['HPlayer6']), dm['hp3'] + dm['hp6'], dm['hp3'])
   

**home player 4**

- if home player in position 4 played in any games at position 1, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 4 and 1. If home player in position 4, didn't play in any other position, the total value of events in position 4 he participated in, are listed.

In [185]:
dm['thp4'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer4'] == dm['HPlayer1']), dm['hp4'] + dm['hp1'], dm['hp4'])
   

- if home player in position 4 played in any games at position 2, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 4 and 2. If home player in position 4, didn't play in any other position, the total value of events in position 4 he participated in, are listed.

In [186]:
dm['thp4'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer4'] == dm['HPlayer2']), dm['hp4'] + dm['hp2'], dm['hp4'])
   

- if home player in position 4 played in any games at position 3, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 4 and 3. If home player in position 4, didn't play in any other position, the total value of events in position 4 he participated in, are listed.

In [187]:
dm['thp4'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer4'] == dm['HPlayer3']), dm['hp4'] + dm['hp3'], dm['hp4'])
   

- if home player in position 4 played in any games at position 5, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 4 and 5. If home player in position 4, didn't play in any other position, the total value of events in position 4 he participated in, are listed.

In [188]:
dm['thp4'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer4'] == dm['HPlayer5']), dm['hp4'] + dm['hp5'], dm['hp4'])
   

- if home player in position 4 played any games in position 6 throughtout the season, the total of events he was on the ice for are calculated with the use of **np.where**. A player might play in position 6 (goaltender position) when his team is trailing in the final minutes of a game and they decide to pull the goaltender to add an additonal skater. It is the sum of  events for position 4 and 6.

In [189]:
dm['thp4'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer4'] == dm['HPlayer6']), dm['hp4'] + dm['hp6'], dm['hp4'])
   

**home player 5**

- if home player in position 5 played in any games at position 1, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 5 and 1. If home player in position 5, didn't play in any other position, the total value of events in position 5 he participated in, are listed.

In [190]:
dm['thp5'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer5'] == dm['HPlayer1']), dm['hp5'] + dm['hp1'], dm['hp5'])
   

- if home player in position 5 played in any games at position 2, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 5 and 2. If home player in position 5, didn't play in any other position, the total value of events in position 5 he participated in, are listed.

In [191]:
dm['thp5'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer5'] == dm['HPlayer2']), dm['hp5'] + dm['hp2'], dm['hp5'])
   

- if home player in position 5 played in any games at position 3, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 5 and 3. If home player in position 5, didn't play in any other position, the total value of events in position 5 he participated in, are listed.

In [192]:
dm['thp5'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer5'] == dm['HPlayer3']), dm['hp5'] + dm['hp3'], dm['hp5'])
   

- if home player in position 5 played in any games at position 4, the total of events he was on the ice for are calculated with the use of **np.where**. It is the sum of events for position 5 and 4. If home player in position 5, didn't play in any other position, the total value of events in position 5 he participated in, are listed.

In [193]:
dm['thp5'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer5'] == dm['HPlayer4']), dm['hp5'] + dm['hp4'], dm['hp5'])
   

- if home player in position 5 played any games in position 6 throughtout the season, the total of events he was on the ice for are calculated with the use of **np.where**. A player might play in position 6 (goaltender position) when his team is trailing in the final minutes of a game and they decide to pull the goaltender to add an additonal skater. It is the sum of  events for position 5 and 6.

In [194]:
dm['thp5'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer5'] == dm['HPlayer6']), dm['hp5'] + dm['hp6'], dm['hp5'])
   

**home player 6**

Position 6 is the goaltender position. A goalie doesn't play in any other position, therefore :

In [195]:
dm['thp6'] = dm['hp6']

## overall player contribution

The impact of each player has been calculated only for his team being at home or away for the season, since home event value and visitor event value were used. The **total contribution** of each player is the total of events he participated for a whole season. Thus, the sum of both home and away event values must be computed.

- create a variable will add up the home event value and away event value for all players of a given team, that played in position 1.

In [196]:
dm['plyr1'] = np.where((dm['HTeamCode'] == dm['VTeamCode']) & (dm['HPlayer1'] == dm['VPlayer1']), dm['thp1'] + dm['tvp1'], dm['thp1'])
   

In [197]:
dm['plyr1'] = np.where((dm['VTeamCode'] == dm['HTeamCode']) & (dm['VPlayer1'] == dm['HPlayer1']), dm['tvp1'] + dm['thp1'], dm['tvp1'])
   

- create a variable that will add up the home event value and away event value for all players of a given team, that played in position 2.

In [198]:
dm['plyr2'] = np.where((dm['HTeamCode'] == dm['VTeamCode']) & (dm['HPlayer2'] == dm['VPlayer2']), dm['thp2'] + dm['tvp2'], dm['thp2'])
   

In [199]:
dm['plyr2'] = np.where((dm['VTeamCode'] == dm['HTeamCode']) & (dm['VPlayer2'] == dm['HPlayer2']), dm['tvp2'] + dm['thp2'], dm['tvp2'])
   

- create a variable that will add up the home event value and away event value for all players of a given team, that played in position 3.

In [200]:
dm['plyr3'] = np.where((dm['HTeamCode'] == dm['VTeamCode']) & (dm['HPlayer3'] == dm['VPlayer3']), dm['thp3'] + dm['tvp3'], dm['thp3'])
   

In [201]:
dm['plyr3'] = np.where((dm['VTeamCode'] == dm['HTeamCode']) & (dm['VPlayer3'] == dm['HPlayer3']), dm['tvp3'] + dm['thp3'], dm['tvp3'])
   

- create a variable that will add up the home event value and away event value for all players of a given team, that played in position 4.

In [202]:
dm['plyr4'] = np.where((dm['HTeamCode'] == dm['VTeamCode']) & (dm['HPlayer4'] == dm['VPlayer4']), dm['thp4'] + dm['tvp4'], dm['thp4'])
   

In [203]:
dm['plyr4'] = np.where((dm['VTeamCode'] == dm['HTeamCode']) & (dm['VPlayer4'] == dm['HPlayer4']), dm['tvp4'] + dm['thp4'], dm['tvp4'])
   

- create a variable that will add up the home event value and away event value for all players of a given team, that played in position 5.

In [204]:
dm['plyr5'] = np.where((dm['HTeamCode'] == dm['VTeamCode']) & (dm['HPlayer5'] == dm['VPlayer5']), dm['thp5'] + dm['tvp5'], dm['thp5'])
   

In [205]:
dm['plyr5'] = np.where((dm['VTeamCode'] == dm['HTeamCode']) & (dm['VPlayer5'] == dm['HPlayer5']), dm['tvp5'] + dm['thp5'], dm['tvp5'])
   

- create a variable that will add up the home event value and away event value for all players of a given team, that played in position 6.

In [206]:
dm['plyr6'] = np.where((dm['HTeamCode'] == dm['VTeamCode']) & (dm['HPlayer6'] == dm['VPlayer6']), dm['thp6'] + dm['tvp6'], dm['thp6'])
   

In [207]:
dm['plyr6'] = np.where((dm['VTeamCode'] == dm['HTeamCode']) & (dm['VPlayer6'] == dm['HPlayer6']), dm['tvp6'] + dm['thp6'], dm['tvp6'])
   

The total contribution of each player per position for the duration of a season has been measured.

## store player evaluation data frame

the player evaluation data frame will be stored and used for the next stage of analysis, player allocation.

In [208]:
dm.to_csv('plyreval.csv', index='False', sep=',')

The next step is to allocate players to their respectful roster position.